In [1]:
print("Ok")

Ok


In [5]:
%pwd

'd:\\repos\\medicalChatBot'

In [2]:
import os
os.chdir("../")

In [7]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
# Extract data from PDF file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents = loader.load()
    
    return documents

In [10]:
extracted_data = load_pdf_file(data="data/")

In [12]:
#extracted_data

In [13]:
def text_split(extracted_data):
    text_splitter =RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [14]:
text_chuncks = text_split(extracted_data)
print("Length of Text Chunks",len(text_chuncks))

Length of Text Chunks 5860


In [17]:
# text_chuncks

In [18]:
from langchain.embeddings import HuggingFaceEmbeddings

In [19]:
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [26]:
embeddings = download_hugging_face_embedding()

d:\repos\medicalChatBot\.venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shoai\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [28]:
query_results = embeddings.embed_query("Hello world")
print("length is ",(query_results))

length is  [-0.03447723016142845, 0.031023241579532623, 0.006735020782798529, 0.026109028607606888, -0.039362046867609024, -0.16030247509479523, 0.06692402064800262, -0.006441468372941017, -0.04745054990053177, 0.01475888304412365, 0.07087527960538864, 0.05552754923701286, 0.019193338230252266, -0.026251371949911118, -0.010109517723321915, -0.02694053389132023, 0.022307438775897026, -0.022226599976420403, -0.1496926248073578, -0.017493126913905144, 0.007676322478801012, 0.054352279752492905, 0.003254470182582736, 0.03172598034143448, -0.08462142199277878, -0.02940603718161583, 0.05159568414092064, 0.048124056309461594, -0.0033148089423775673, -0.058279216289520264, 0.041969288140535355, 0.022210724651813507, 0.1281888484954834, -0.022338956594467163, -0.011656180024147034, 0.06292832642793655, -0.03287629038095474, -0.09122608602046967, -0.03117540292441845, 0.052699580788612366, 0.0470348559319973, -0.08420306444168091, -0.03005615621805191, -0.02074478752911091, 0.009517795406281948,

In [30]:
from pinecone.grpc import PineconeGRPC as PineCone
from pinecone import ServerlessSpec
from dotenv import load_dotenv

load_dotenv()
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

pc = PineCone(api_key=PINECONE_API_KEY)

index_name= "medical-chat-bot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [41]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["PINECONE_API_KEY"]= os.environ.get("PINECONE_API_KEY")

In [33]:
# Embed each chunk and upsert the embeddings into your Pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chuncks,
    index_name=index_name,
    embedding= embeddings
)

In [35]:
# Load existing index

from langchain_pinecone import PineconeVectorStore

# Embed each chunk and upsert the embeddings into your Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [36]:
docsearch

In [37]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [38]:
retriever_docs = retriever.invoke("What is Acne?")

In [39]:
retriever_docs

[Document(id='779499a8-c6ff-4919-a052-b5bf4d06b3c6', metadata={'page': 39.0, 'page_label': '40', 'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='9dd9dd13-2495-447e-b2f1-228de42fb9e2', metadata={'page': 38.0, 'page_label': '39', 'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='287b20fd-9dca-47f1-977c-42cdbb8076b2', metadata={'page': 37.0, 'page_label': '38', 'source': 'data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characteri

In [54]:
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="llama3.2")

In [55]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following peices of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [56]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [67]:
response = rag_chain.invoke({"input":"What is oxygen?"})
print(response["answer"])

I don't know what oxygen is specifically. According to the provided context, oxygen is not dissolved in the blood but instead combines with hemoglobin in red blood cells, and its amount is measured as part of the oxygen content and oxygen saturation. The value of oxygen saturation compares the actual combined oxygen with the total capacity of hemoglobin.


In [68]:
response = rag_chain.invoke({"input":"Who is Gale Encyclopedia of Medicine"})

response

{'input': 'Who is Gale Encyclopedia of Medicine',
 'context': [Document(id='a1484767-871b-4a62-9f72-5f6eb6348f2e', metadata={'page': 1.0, 'page_label': '2', 'source': 'data\\Medical_book.pdf'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION'),
  Document(id='8906bc81-932f-4f20-a7b0-d45d59449cb7', metadata={'page': 6.0, 'page_label': '7', 'source': 'data\\Medical_book.pdf'}, page_content='point the reader to related entries in the encyclopedia.\nGALE ENCYCLOPEDIA OF MEDICINE 2 IX\nINTRODUCTION'),
  Document(id='3f0fedc5-774c-4d38-9c7d-465d66f95f4d', metadata={'page': 7.0, 'page_label': '8', 'source': 'data\\Medical_book.pdf'}, page_content='• Resources section directs users to additional sources\nof medical information on a topic.\n• A comprehensive general index allows users to easily\ntarget detailed aspects of any topic, including Latin\nnames.\nGRAPHICS\nThe Gale Encyclopedia of Medicine 2 is enhanced\nwith over 675 color images, including photos, charts,\ntables,